# Lab Assignment Two: Exploring Image Data

***Christopher Cook, Bonita Davis, Anekah Kelley, Davis Lynn***

## 1. Preparation and Overview

### **1.1 Business Understanding**

This is a dataset of 15,149 songs from 3,083 artists over the course of over a century (1899-2024).  It is sorted by 18 different features:
*    Track (string)
*    Artist (string)
*    Year (int)
*    Duration (int, in ms)
*    Time Signature (int, numerator of time signature; i.e. 4 means 4/4)
*    Danceability (float, 0-10)
*    Energy (float 0-1)
*    Key (int, 0-11; 0 = C, etc.)
*    Loudness (float, loudness in db -47.4=0.92)
*    Mode (int, major/minor modality; 0/1)
*    Speechiness (float, 0-1)
*    Acousticness (float, 0-1)
*    Instrumentalness (float 0-1)
*    Liveness (float, 0-1)
*    Valence (float, 0-1)
*    Tempo (tempo, BPM from 0-220)
*    Popularity (int, 0-100)
*    Genre (string)

This data was collected directly from the Spotify API, allowing for additional attributes such as Loudness, Speechiness, etc. The dataset is free to download on Kaggle through the link below.

Data Source: https://www.kaggle.com/datasets/thebumpkin/10400-classic-hits-10-genres-1923-to-2023/data

The task of this dataset is to serve as a touchpoint for individuals in the music industry to explore trends, compare genres, analyze sonic qualities, and see how classic hits across time periods and genres compare to once another (Kaggle).  The sheer size of the dataset, along with the amount of information provided about each song, makes this an excellent resource to understand musical trends.  It can also be used to see how an individual artist or genre has performed over time, providing insights into how future songs/albums might perform.

This is the primary business-case of this dataset and our classifier -- attempting to predict attributes about songs.  This information can be used by companies like Spotify and Apple Music to predict the likelihood that a user will like a song based on the attributes of the songs they have been previously shown to like (if they typically like pop music with a loudness of 97, that information can be used to predict new songs that they may also like).  These businesses care more about the relationships between all of the attributes, as it is the combination of these attributes that creates a listening style.  This information could also be used by a music producer or musician to determine what attributes about songs in different genres/time periods tend to make them popular in order to see how to best leverage the artist's skills to create a hit song.  These businesses care more about the interplay between the attributes that they deem to be relevant based on the artist and popularity, as that gives the best blueprint for success.

This model would likely be deployed, although it may differ depending on the user.  Users like Spotify, Apple Music, YouTube Music, and other such companies would use this data constantly, making it a deployed model.  In contrast, while some music producers may consistently run new models, it is possible that it could be semi-offline and just generated once for the relevant artists.  This data was likely collected with these intentions in mind, as classifiers could be trained and sold to such businesses for a sizeable amount of money.

The use of such a classifier by companies like Spotify and Apple Music is supported by the fact that Spotify currently uses a similar classifier to create their "Made For You" playlist provided to users (https://www.music-tomorrow.com/blog/how-spotify-recommendation-system-works-a-complete-guide-2022).  They use very similar attributes (which is the case because our dataset directly incorporates Spotify information) to predict the likelihood that a user will enjoy a song based on their listening history.  Spotify's algorithm is many times more complex than anything we would be capable of creating at this point in time, so holding ourselves to that standard would be simply foolish.  Additionally, Spotify does not publish an "accuracy" rate for predicting songs.  Further complicating the problem, an NIH study showed that roughly 15% of people skip songs that they haven't listened to, before even listening to it, and that others skipped the songs shortly after starting them if they didn't quickly enjoy the song (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004375/).  While this is a relatively small percentage, it shows the importance of a very accurate classifier.  Ideally, a classifier with a roughly 90% accuracy is likely possible with advanced algorithms, predicts songs with an acceptable margin of error, and will result in benefits for companies.

For music producers and artists interested in predicting the success of their songs, a much lower bar of accuracy is acceptable.  Similar classifier algorithms boast an accuracy of anywhere between 55-77% (https://cs229.stanford.edu/proj2015/140_report.pdf, https://towardsdatascience.com/song-popularity-predictor-1ef69735e380, https://www.nature.com/articles/s41598-022-25430-9).  In truth, anything able to predict with an accuracy greater than 50% would be considered successful to some degree, as popularity is highly subjective, changes dramatically over time, and is often a product of purely qualitative features that are difficult to connect to quantitative information, despite best efforts.  Aiming for an accuracy of roughly 65% would put our model squarely in the middle of currently available algorithms, making it both realistic and acceptable in the industry.

### 1.2 Preparation of Data

In [2]:
import pandas as pd 

# find the data type
df = pd.read_csv('ClassicHit.csv')

print(df.info())

ModuleNotFoundError: No module named 'pandas'

Based on the dataframe information, there are no missing values in the dataset (there are 15150 rows in the data, and 15150 entries in each feature).  Furthermore, the values for the data given are reasonable, as can be seen through the range given in Kaggle.  This range shows the highest and lowest values in the dataset, and all values in these ranges are reasonable, meaning that all values in the dataset are reasonable.

If there were missing values in the dataset, we could use mean or median imputation for numerical data (year, duration, danceability, energy, etc.), and mode for the categorical features (genre. etc.).

In [1]:
#data summary
df.describe()

NameError: name 'df' is not defined

In [ ]:
data_des = pd.DataFrame()

data_des['Attributes'] = df.columns
data_des['Description'] = ['Name of song', 'Name of singer', 'Year released', 'Length of song in milliseconds', 'Musical time signature',
        'Suitability of dancing (0-1)', 'Intensity of the song (0-1)', 'Key of the track (0-11)', 'Loudness of track in dB', 'Modality (major or minor)',
        'Measures the presence of spoken words (0-1)', 'Measure of how acoustic the track is (0-1)', 'Measure of likelihood that track is instrumental (0-1)',
        'Measure of the presence of a live audience (0-1)', 'Measure of musical positiveness (0-1)', 'Speed of the track (BPM)', 'Track popularity (0-100)',
        'Musical genre of the song']
data_des['Range'] = ['N/A', 'N/A', '1899-2024', '23.0k-3.06m', '3 or 4', '0.0-1.0', '0.0-1.0', '0-11', '-47.4-0.92', '0 or 1', '0.0-1.0', '0.0-1.0', '0.0-1.0', 
        '0.0-1.0', '0.0-1.0', '0-220', '0-100', 'N/A']

data_des

This table summarizes the attributes found in this dataset.  As previously stated, there are no missing values in the dataset.

We should check whether there are duplicate values:

In [ ]:
#check for duplicates
idx = df.duplicated()
len(df[idx])

This means that there is one duplicated song. We should see which it is:

In [ ]:
#check which is a duplicate
df[idx]

In [ ]:
# Running duplicate code again to double check
#check for duplicates
idx = df.duplicated()
len(df[idx])

This is a complete duplicate -- "Happy Now" by Zedd, released in 2019.  After verifying that it was a perfect duplicate, our group chose to delete the song from our dataset. This completed the cleaning of our data, as we ensured that there was no missing data and there are now no duplicates.

The only issue with the data quality is we do not know how many of the features are defined or quantified, such as popularity or loudness. It is unknown whether these features that were provided to us are reliable or consistent. The following features I would consider unknown whether or not they are truly reliable: danceability, energy, loudness, speechiness, instrumentalness, liveness, valence. Another issue is that songs can alternate between key, mode, and time signature. Furthermore only 2 of the 7 modes are listed in dataset (major and minor are present, but not dorian, phrygian, lydian, mixolydian, or locrian). Simply speaking, there are complexities in music lost when only giving one value to key, mode, and time signature.

### REMOVE FIRST PARAGRAPH IF WE DON'T END UP USING ERA/ERA2
We created a new division in the data called "era", which is described as year values < 1982 and year values > 1982. This division helps us visualize how the music changed over time in an easily digestible manner. It also marks the beginning of widespread use of synthesizers in music. We also created a second division in the data, called "era2". This variable breaks the music into 6 different time ranges: -1945, 1946-1960, 1961-1975, 1976-1990, 1991-2008, and 2009-. These two variables were extremely useful for data visualization, as it was otherwise nearly impossible to use the year variable.

This data has been shown to be ready for our classification model, as it has no duplicates, no missing values, and all values are within a reasonable range.  We are also aware of which values are numeric and non-numeric, an important piece of information as we begin calculations that may require numeric values in order to predict information.

### I'm not entirely sure if we need to add a breakdown of variables because I don't think we technically did any pre-processing, but I'll ask.

### **1.3 Training/Testing Data Split**

Our team elected to use the standard 80% training, 20% testing split for our data.  This split is appropriate for several reasons.
1.   Our data contains information about over 15,000 songs, making the 80% of training data represent over 12,000 songs.  This is a more than adequate amount of data to train our classifier and provide it with many different combinations of attributes.
2.   The size of our dataset also means that the 20% testing data represents over 3,000 songs, making it a large enough chunk of data that we can provide reasonable accuracies that reflect the actual accuracy of the classifier.
3.   This split is the the industry standard for datasets, as it has been proven time and time again that a split of roughly 70-80% training and 20-30% testing provides the best resuls (https://scholarworks.utep.edu/cs_techrep/1209/)

In [ ]:
#put division of data here, done in modeling (just move the chunk)

## 2. Modeling

In [5]:
import numpy as np
from scipy.special import expit
from sklearn.model_selection import train_test_split
import pandas as pd
import plotly
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_intercept(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self, X, add_intercept=True):
        # add bias term if requested
        Xb = self._add_intercept(X) if add_intercept else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction

In [ ]:
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'


    @property
    def coef_(self):
        if(hasattr(self,'w_')):
            return self.w_[1:]
        else:
            return None

    @property
    def intercept_(self):
        if(hasattr(self,'w_')):
            return self.w_[0]
        else:
            return None

        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_intercept=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_intercept(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

In [ ]:
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self, X, y):
        # Ensure y is numeric, converting categorical to integers or floats
        y = y.astype('float')  # or y = y.astype('int') if appropriate
    
        ydiff = y - self.predict_proba(X, add_intercept=False).ravel()  # Get y difference
        ydiff = np.array(ydiff)  # Convert ydiff to a NumPy array to allow multi-dimensional indexing
        gradient = np.mean(X * ydiff[:, np.newaxis], axis=0)  # Multiply ydiff as a column vector

        return gradient.reshape(self.w_.shape)

In [ ]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    @property
    def coef_(self):
        if(hasattr(self,'w_')):
            return self.w_[:,1:]
        else:
            return None

    @property
    def intercept_(self):
        if(hasattr(self,'w_')):
            return self.w_[:,0]
        else:
            return None
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,
                                                 self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row

In [ ]:
df = pd.read_csv('ClassicHit.csv')

#This is how I (Chris) believe the train/test data should be done

Xcoloumns = ['Year', 'Duration', 'Time_Signature', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']

X = df[Xcoloumns]
y = pd.cut(df['Popularity'], bins=[-float('inf'), 15, 75, float('inf')], labels=[0, 1, 2]) #I am choosing the splits to determine whether a song will be, not populary at all, mediocre, very popular
#I chose this way so we can find something meaningful no matter what the classification is for the songs
#y = (df['Popularity'] > 75).astype(int) #Testing for binary case

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size=0.2)

plotly.offline.init_notebook_mode() # run at the start of every notebook

graph1 = {'x': np.unique(y),
          'y': np.bincount(y)/len(y),
            'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Class Distribution',
                'autosize':False,
                'width':400,
                'height':400}

plotly.offline.iplot(fig)

In [ ]:
lr = LogisticRegression(0.1,500)
lr.fit(X_train,y_train)
print(lr)

yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

## 3. Deployment

## 4. Exceptional Work

In [ ]:
import numpy as np

# Define the softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability trick
    return exp_z / exp_z.sum(axis=1, keepdims=True)

# Mean Squared Error Loss Function for multi-class
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Gradient of the MSE Loss
def gradient_mse(X, y_true, y_pred):
    return -(2 / X.shape[0]) * np.dot(X.T, (y_pred - y_true))  # Ensure shapes align

# Hessian of the MSE Loss
def hessian_mse(X, y_pred):
    # Calculate the diagonal matrix for each class
    diag = y_pred * (1 - y_pred)  # Shape (num_samples, num_classes)
    
    # Now we need to calculate the Hessian for each class
    H = np.zeros((X.shape[1], X.shape[1]))  # Initialize a 2D Hessian
    
    for i in range(y_pred.shape[1]):  # Iterate over each class
        H += (1 / X.shape[0]) * np.dot(X.T, diag[:, i].reshape(-1, 1) * X)  # Properly scale and shape

    return H  # Return the Hessian (summed over classes)

def newtons_method(X, y_true, w, num_iterations, tolerance=1e-6):
    for i in range(num_iterations):
        # Compute predictions using current weights
        y_pred = softmax(np.dot(X, w))  # Softmax on the dot product
        
        # Compute gradient and Hessian for all classes
        grad = gradient_mse(X, y_true, y_pred)
        H = hessian_mse(X, y_pred)

        # Add regularization to the Hessian
        H_inv = np.linalg.inv(H + np.eye(H.shape[0]) * 1e-5)

        # Update weights using Newton's method
        w_new = w - np.dot(H_inv, grad)

        # Check for convergence
        if np.linalg.norm(w_new - w) < tolerance:
            print(f'Converged after {i + 1} iterations')
            break

        # Update weights
        w = w_new

    return w

This code implements an optimization technique for logistic regression using mean square error as the objective function (in this case, a loss function).  It defines softmax, used to calculate the predicted probabilities (because this is multi-class classification), and an MSE loss function to measure the error.  The code then calculates the gradient and Hessian of the MSE loss, allowing it to update the model's weights.  It then uses Newton's Method, iteratively updating the weights until convergence is achieved or max iterations are reached.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

# Load data
dataframe = pd.read_csv('ClassicHit.csv')

# Select columns for features
X_columns = ['Year', 'Duration', 'Time_Signature', 'Danceability', 'Energy', 
             'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 
             'Instrumentalness', 'Liveness', 'Valence', 'Tempo']
X = dataframe[X_columns].astype(float)

# Create target variable using pd.cut
y = pd.cut(dataframe['Popularity'], bins=[-float('inf'), 15, 75, float('inf')], labels=[0, 1, 2]).astype(int)

# Check class distribution
print(y.value_counts())

# Convert y to NumPy array and reshape for OneHotEncoder
y = y.values.reshape(-1, 1)  # Reshape to 2D array

# One-hot encode the target variable
encoder = OneHotEncoder(sparse_output=False)
y_one_hot = encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y_one_hot, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the model (not using scikit-learn here since we are implementing Newton's method)
# Define the number of classes
num_classes = y_one_hot.shape[1]
num_features = X.shape[1]

# Initialize weights randomly for Newton's method
w = np.random.randn(num_features, num_classes)  # Weights shape: (num_features, num_classes)

# Train the model using the Newton's method defined earlier
w_trained = newtons_method(X_train, y_train, w, num_iterations=100, tolerance=1e-6)

# Make predictions on the test set
y_pred_prob = softmax(np.dot(X_test, w_trained))  # Get predicted probabilities
y_pred = np.argmax(y_pred_prob, axis=1)  # Get class labels by taking the argmax

# Convert y_test back to class labels
y_test_labels = np.argmax(y_test, axis=1)

# Calculate Mean Squared Error (if desired)
mse = mse_loss(y_test_labels, y_pred)  # Both are now 1D arrays
rmse = np.sqrt(mse)

print(f"Mean Squared Error on Test Set: {mse}")
print(f"Root Mean Squared Error on Test Set: {rmse}")

This section of the code uses the previously defined functions and applies them to the dataset we selected.  I elected to use the same relevant features as we did previously, allowing a direct comparison between the two models.  Similarly, the "Popularity" column is split into three classes (not popular, mediocre, and very popular), allowing for better classification.  This variable is one-hot encoded, and then split into training and testing data.  I standardized the features in order to have better model performance, and randomly assigned weights for the model and then trained it using Newton's method to optimize the weights based on the training data.  It then predicts the test set, converts them to class labels, and calculates the MSE and RMSE as examples of performance.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred_prob = softmax(np.dot(X_test, w_trained))  # Get predicted probabilities
y_pred = np.argmax(y_pred_prob, axis=1)  # Get class labels by taking the argmax

# Convert y_test back to class labels (y_test is already one-hot encoded)
y_test_labels = np.argmax(y_test, axis=1)  # Ensure y_test is one-hot encoded before this

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')
f1 = f1_score(y_test_labels, y_pred, average='weighted')

print(f"Accuracy on Test Set: {accuracy}")
print(f"Precision on Test Set: {precision}")
print(f"Recall on Test Set: {recall}")
print(f"F1 Score on Test Set: {f1}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
cm = confusion_matrix(y_test_labels, y_pred)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1, 2], yticklabels=[0, 1, 2])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC for each class
n_classes = y_one_hot.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_labels, y_pred_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting the ROC curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve for class {i} (area = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

These graphs show that this classification method is not particularly effective -- it frequently incorrectly predicts that mediocre songs are going to be unpopular, and doesn't predict nearly as many songs to be very popular as there actually are.  This could be for a number of reasons, including that the classes are not weighted, leading to a prediction of 0, followed by 1, followed by 2 to be the most common result, regardless of other factors.  It may also mean that there is such little correlation between the features that the model cannot accurately predict the class.

Because of this issue, I decided to try implementing a weighted version of this code, seen below.

In [ ]:
import numpy as np

# Define the softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability trick
    return exp_z / exp_z.sum(axis=1, keepdims=True)

# Mean Squared Error Loss Function for multi-class with class weights
def weighted_mse_loss(y_true, y_pred, class_weights):
    # Reshape class_weights to match the shape of y_true and y_pred
    class_weights_matrix = class_weights.reshape(1, -1)  # Shape (1, num_classes)
    weighted_squared_errors = class_weights_matrix * (y_true - y_pred) ** 2
    return np.mean(np.sum(weighted_squared_errors, axis=1))  # Average over all samples

# Gradient of the weighted MSE Loss
def gradient_weighted_mse(X, y_true, y_pred, class_weights):
    # Calculate weighted errors; ensure proper broadcasting
    weighted_errors = (y_pred - y_true)  # Shape (num_samples, num_classes)
    return -(2 / X.shape[0]) * np.dot(X.T, weighted_errors * class_weights)  # Apply class weights

# Hessian of the weighted MSE Loss
def hessian_weighted_mse(X, y_pred, class_weights):
    # Expand class_weights for each sample
    expanded_weights = class_weights.reshape(1, -1)  # Shape (1, num_classes)
    diag = (expanded_weights * y_pred * (1 - y_pred)).reshape(-1, y_pred.shape[1])  # Shape (num_samples, num_classes)

    # Initialize a 2D Hessian
    H = np.zeros((X.shape[1], X.shape[1]))

    for i in range(y_pred.shape[1]):  # Iterate over each class
        H += (1 / X.shape[0]) * np.dot(X.T, diag[:, i].reshape(-1, 1) * X)  # Properly scale and shape

    return H  # Return the Hessian (summed over classes)

def newtons_method(X, y_true, w, num_iterations, tolerance=1e-6, class_weights=None):
    for i in range(num_iterations):
        # Compute predictions using current weights
        y_pred = softmax(np.dot(X, w))  # Softmax on the dot product
        
        # Compute gradient and Hessian for all classes with class weights
        grad = gradient_weighted_mse(X, y_true, y_pred, class_weights)
        H = hessian_weighted_mse(X, y_pred, class_weights)

        # Add regularization to the Hessian
        H_inv = np.linalg.inv(H + np.eye(H.shape[0]) * 1e-5)

        # Update weights using Newton's method
        w_new = w - np.dot(H_inv, grad)

        # Check for convergence
        if np.linalg.norm(w_new - w) < tolerance:
            print(f'Converged after {i + 1} iterations')
            break

        # Update weights
        w = w_new

    return w

This code uses weighted MSE loss instead of standard MSE, allowing for a different importance to be assigned to classes based on frequency, hopefully helping with issues related to class imbalance.  My calculations for gradient and Hessian also incorporate class weights, ensuring that the updates are influenced by the class distribution in the training set.  The code remains overall similar, but emphasizes the importance of class weight

In [ ]:
# Load data
dataframe = pd.read_csv('ClassicHit.csv')

# Select columns for features
X_columns = ['Year', 'Duration', 'Time_Signature', 'Danceability', 'Energy', 
             'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 
             'Instrumentalness', 'Liveness', 'Valence', 'Tempo']
X = dataframe[X_columns].astype(float)

# Create target variable using pd.cut
y = pd.cut(dataframe['Popularity'], bins=[-float('inf'), 15, 75, float('inf')], labels=[0, 1, 2]).astype(int)

# Check class distribution
print(y.value_counts())

# Convert y to NumPy array
y = y.values.reshape(-1, 1)  # Reshape to 2D array for OneHotEncoder

# One-hot encode the target variable
encoder = OneHotEncoder(sparse_output=False)
y_one_hot = encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y_one_hot, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the number of classes
num_classes = y_one_hot.shape[1]
num_features = X.shape[1]

# Initialize weights randomly for Newton's method
w = np.random.randn(num_features, num_classes)  # Weights shape: (num_features, num_classes)

# Class weights based on the inverse frequency
class_counts = np.sum(y_train, axis=0)  # Count instances of each class
total_instances = y_train.shape[0]

# Avoid division by zero and ensure we handle missing classes
class_counts = np.clip(class_counts, a_min=1, a_max=None)  # Prevent division by zero
class_weights = total_instances / (num_classes * class_counts)  # Shape (num_classes,)

# Train the model using the modified Newton's method with class weights
w_trained = newtons_method(X_train, y_train, w, num_iterations=100, tolerance=1e-6, class_weights=class_weights)

# Make predictions on the test set
# Make predictions on the test set
y_pred_prob = softmax(np.dot(X_test, w_trained))  # Get predicted probabilities
y_pred = np.argmax(y_pred_prob, axis=1)  # Get class labels by taking the argmax

# Convert y_test back to class labels (y_test is already one-hot encoded)
y_test_labels = np.argmax(y_test, axis=1)  # Ensure y_test is one-hot encoded before this

# Convert y_pred to one-hot encoded format
y_pred_one_hot = np.zeros((y_pred.shape[0], num_classes))  # Shape (num_samples, num_classes)
y_pred_one_hot[np.arange(y_pred.shape[0]), y_pred] = 1  # One-hot encode predictions

# Calculate Mean Squared Error (if desired)
mse = weighted_mse_loss(y_test_labels, y_pred_one_hot, class_weights)  # Use the one-hot encoded predictions
rmse = np.sqrt(mse)

print(f"Mean Squared Error on Test Set: {mse}")
print(f"Root Mean Squared Error on Test Set: {rmse}")

The analysis of the dataset revealed a significant class imbalance in the target variable, Popularity, categorized into three classes: low (0), medium (1), and high (2), with 12,628 instances in Class 1, 1,693 in Class 0, and 828 in Class 2. A modified Newton's method was employed for training a multi-class logistic regression model, using class weights to address the imbalance. The model's performance was evaluated with a Mean Squared Error (MSE) of 4.93 and a Root Mean Squared Error (RMSE) of 2.22, indicating reasonable predictive accuracy, but suggesting the need for further strategies to handle class imbalance and enhance model evaluation metrics.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred_prob = softmax(np.dot(X_test, w_trained))  # Get predicted probabilities
y_pred = np.argmax(y_pred_prob, axis=1)  # Get class labels by taking the argmax

# Convert y_test back to class labels (y_test is already one-hot encoded)
y_test_labels = np.argmax(y_test, axis=1)  # Ensure y_test is one-hot encoded before this

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')
f1 = f1_score(y_test_labels, y_pred, average='weighted')

print(f"Accuracy on Test Set: {accuracy}")
print(f"Precision on Test Set: {precision}")
print(f"Recall on Test Set: {recall}")
print(f"F1 Score on Test Set: {f1}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
cm = confusion_matrix(y_test_labels, y_pred)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1, 2], yticklabels=[0, 1, 2])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC for each class
n_classes = y_one_hot.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_labels, y_pred_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting the ROC curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve for class {i} (area = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

Overall, the confusion matrices for both implementations show that this is not a particularly effective classification method, as the non-weighted version has a high chance of a false negative, while the weighted version barely ever predicts 1s.  I attempted to fix this by manually weighting the 1 column (multiplying the values by anything from 3-50), but this resulted in an extremely high rate of false positives.  Given our business-case, false positives are the exact opposite of what we would want, meaning that if we were to select either of these cases, the non-weighted implementation would be better.  However, neither method is particularly helpful, as they have an accuracy of either roughl 35% or roughly 4%.

I prefer maximum likelihood for several reasons.  For one, with this dataset, maximum likelihood results in a much higher accuracy rate than minimum mean-square error.
### Complete in the morning